# Mean spectral data preparation of wheat species for 40 varieties

In [ ]:
import spectral.io.envi as envi
import cv2
from spectral import envi
from spectral import *
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

import tifffile as t
import pandas as pd
from skimage import measure
from skimage import color, morphology
from skimage.color import label2rgb
import pandas as pd

In [ ]:
coordinates_CU=[]
coordinates_CD=[]

In [ ]:
def show_img(img): # for displaying the hyperspectral image
    fig = plt.figure(figsize = (8,8))
    plt.imshow(img)
    plt.show()

In [ ]:
def preprocess(img1): # binary thresholding technique for generating the binary mask
    median = cv2.medianBlur(img1, 5)
    # applying thresholding to filtered image by taking threshold value 0.19 and displaying the image
    for i in range(median.shape[0]):
        for j in range(median.shape[1]):
            if median[i][j]>0.19:
                median[i][j]=1
            else:
                median[i][j]=0
    return median

In [ ]:
def img_multiply(img,median): #multiply binary mask with all spectral bands for extracting the ROI 
    img_result=[]
    for i in range(14,161):  #(14,161)
        y=np.multiply(np.squeeze(img[25:800,:,i]),median)
        img_result.append(y)
    return img_result

In [ ]:
def img_coord(median): # for finding the centre and coordinates of each seeds of the ROI
    label_img=measure.label(median, connectivity=median.ndim)
    img_label_overlay = label2rgb(label_img, image=median)
    props = measure.regionprops_table(label_img, median, 
                          properties=['centroid','coords'])
    df = pd.DataFrame(props)
    return df

In [ ]:
def dataframe_rearrange(df): # for reordering the seeds to maintain the correspondence between the crease up and crease
                              #down seeds
    l=[]
    for j in range(0,72,6):
        df1=df[j:j+6]
        df2=df1.sort_values(by=['centroid-1'])
        df2.reset_index(drop=True, inplace=True)
        l.append(df2)
    vertical_concat = pd.concat([l[0], l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10],l[11]], axis=0)
    return vertical_concat

In [ ]:
def spectra(img_result,df): # for finding the mean spectral values for each seed 
    spectra_mean=[]
    spectra_mean_var=[]
    coordinates=[]
    coordinates=df['coords']
    #print(coordinates[0][0][1])
    for i in img_result:
        spectra_mean=[]
        for j in coordinates:
            reflectance_val=[]
            for k in j:
                reflectance_val.append(i[k[0]][k[1]])
            x=np.sum(reflectance_val)/len(reflectance_val)
            spectra_mean.append(x)
        spectra_mean_var.append(spectra_mean)
    df=pd.DataFrame(spectra_mean_var)
    df_transpose=df.transpose()
    return df_transpose

In [ ]:
def spectra_cu_arrange(spectra_cu_new): # for reordering the seeds in the plate
    spectra_cu_var=[]
    l=[5,4,3,2,1,0,11,10,9,8,7,6,17,16,15,14,13,12,23,22,21,20,19,18,29,28,27,26,25,24,35,34,33,32,31,30,41,40,39,38,37,36,47,46,45,44,
  43,42,53,52,51,50,49,48,59,58,57,56,55,54,65,64,63,62,61,60,71,70,69,68,67,66]
    for i in l:
        spectra_cu_var.append(spectra_cu_new[i])
    return spectra_cu_var

In [ ]:
def mergeList(lst1, lst2):
    return [sub[item] for item in range(len(lst2))
                      for sub in [lst1, lst2]]

In [ ]:
# to merge all the 14 dataframes of single variety 
def merge_dataframe(l):
    vertical_concat = pd.concat([l[0], l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10],l[11],l[12],l[13]], axis=0)
    return vertical_concat

In [ ]:
# to merge all the 40 dataframe of all the varieties used for experimentation
def merge_dataframe_single(l):
    vertical_concat = pd.concat([l[0], l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10],l[11],l[12],l[13],l[14],l[15],l[16],
                                l[17],l[18],l[19],l[20],l[21],l[22],l[23],l[24],l[25],l[26],l[27],l[28],l[29],l[30],l[31],l[32],
                                l[33],l[34],l[35],l[36],l[37],l[38],l[39]], axis=0)
    return vertical_concat

In [ ]:
coord_CU=[]
coord_CD=[]
l2_cu=[]
l2_cd=[]
coord_cu=[]
spectra_arrange_cu=[]
def variety_process(img,img2):
    img1= img[25:800,:,50]
    median=preprocess(img1)
    img_result=img_multiply(img,median)
    df1= img_coord(median) # for storing the values of coordinates and area
    df_arrange_cu=dataframe_rearrange(df1)
    spectra_cu=spectra(img_result,df_arrange_cu)  # store frame of spectra
    l2_cu.append(spectra_cu)
    img3= img2[25:800,:,50]
    median2=preprocess(img3)
    img_result2=img_multiply(img2,median2)
    df2= img_coord(median2) #receive initial dataframe of coordinates
    df_arrange_cd=dataframe_rearrange(df2) #receive dataframe of arrnaged coordinates
    spectra_cd=spectra(img_result2,df_arrange_cd)
    l2_cd.append(spectra_cd)
    n1=spectra_cu.to_numpy()
    n2=spectra_cd.to_numpy()
    x1=n1.tolist()
    x2=n2.tolist()
    spectra_arrange=spectra_cu_arrange(x1)
    spectra_arrange_cu.append(spectra_arrange)
    x3_merge=mergeList(spectra_arrange,x2)
    df3=pd.DataFrame(x3_merge)
    return(df3)

In [ ]:
variety_name_list_40=['RAJ_MOLIYA_RODHKA_1_R_22','RAJ1482_R_22','RAJ3777_R_22','RAJ4037_R_22','RAJ4079_R_22','RAJ4083_R_22','RAJ4120_R_22','RAJ4238_R_22','RAJ3077_R_22','RAJ3765_R_22','DBW187_L_22','DBW222_L_22',
                      'HD3086_L_22','PBW 803_L_22','WH1105_L_22'
                      ,'UAS428_DH_22','MACS3949_DH_22','MACS4100_DH_22','UAS446_DH_22','HD4728_MP_22','HI8777_MP_22','HI8759_MP_22','HI8737_MP_22','HI8663_MP_22','HI8713_MP_22',
                      'HI8823_MP_22','DDW47_K_22','PDW291_K_22','HI8498(MALAV_SHAKTI)_K_22','A-9-30-1_21_22_K',
                      'DDK1001_21_22_K','DDK1009_21_22_K','DDK1025_21_22_K','DDK1029_21_22_K','MACS2971_21_22_K','NP 200_K_21_22_K_SPECIES',
                      'HW1098_K_21','DT46_21_22_K','TL2908_21_22_K','TL2942_21_22_K']
len(variety_name_list_40)

In [ ]:
dataframe_var_list=[] # for reading and processing the 40 wheat varieties
for i in variety_name_list_40:
    l1=[] #list to store datafram
    for k in range(1,15):
        img_1 = envi.open("D:\\species_wheat_40\\{}\\CU_{}.bil.hdr.".format(i,k),"D:\\species_wheat_40\\{}\\CU_{}.bil".format(i,k))
        img_2 = envi.open("D:\\species_wheat_40\\{}\\CD_{}.bil.hdr.".format(i,k),"D:\\species_wheat_40\\{}\\CD_{}.bil".format(i,k))
        df_var=variety_process(img_1,img_2)
        l1.append(df_var)
    df_merge=merge_dataframe(l1)
    dataframe_var_list.append(df_merge)
